# Dados de Entrada
* Selecione "Adicionar ao Drive"
* Links:
  * https://tinyurl.com/bigdata-gut-pt
  * https://tinyurl.com/bigdata-amz





# Setup

## Instalação de pacotes

In [ ]:
!apt-get update  > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install findspark pyspark
!apt-get install netcat > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e2ec1df90f107fad22c589a86f1ea5e259a5c9d9930ae5ea5a3842b12a46720f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


## Acesso ao Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preparação do ambiente

In [ ]:
%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.5.1-bin-hadoop3

env: PYTHONHASHSEED=1234
env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
env: SPARK_HOME=/content/spark-3.5.1-bin-hadoop3


In [ ]:
import findspark
findspark.init("/content/spark-3.5.1-bin-hadoop3")

In [ ]:
findspark.find()


'/content/spark-3.5.1-bin-hadoop3'

# Streaming de arquivos

## Leitura de dados de stream
O notebook é mais para ser um editor de texto, o código vai rodar em linha de comando

In [ ]:
%%file basic_file_stream.py #essa célula não vai ser executada, ela escreve um arquivo
#se rodasse não veria saída nenhuma
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Big Data Streaming') \
    .getOrCreate()

spark.sparkContext.setLogLevel('WARN')

lines = spark \
    .readStream \
    .format('text') \
    .load('/content/stream_input')

query = lines.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination(120)


Writing basic_file_stream.py


In [ ]:
!mkdir /content/stream_input/
!rm /content/stream_input/*

rm: cannot remove '/content/stream_input/*': No such file or directory


In [ ]:
!spark-submit basic_file_stream.py

## Transformações básicas

In [ ]:
%%file basic_transformations.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Big Data Streaming') \
    .getOrCreate()

spark.sparkContext.setLogLevel('WARN')

# Stream de leitura dos dados.
# Cada linha dos dados de entrada será armazenada em uma coluna chamada line, do tipo String
lines = spark \
    .readStream \
    .schema('line STRING') \
    .format('text') \
    .load('/content/stream_input')

# Converte a coluna 'line' para minuscula, e aplica uma expressão regular para remover caracteres não-letra
# Resultado é um novo dataframe com a coluna 'line_clean'
clean_lines = lines \
    .withColumn('line_lower', lower(col('line'))) \
    .select(regexp_replace('line_lower', r'[^a-zà-ù ]', '').alias('line_clean'))


query = clean_lines.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination(120)


In [ ]:
!rm /content/stream_input/*

In [ ]:
!spark-submit basic_transformations.py

## Contagem de palavras por lote

In [ ]:
%%file wc_per_batch.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Big Data Streaming') \
    .getOrCreate()

spark.sparkContext.setLogLevel('WARN')

lines = spark \
    .readStream \
    .schema('line STRING') \
    .format('text') \
    .load('/content/stream_input')

clean_lines = lines \
    .withColumn('line_lower', lower(col('line'))) \
    .select(regexp_replace('line_lower', r'[^a-zà-ù ]', '').alias('line_clean'))


# Separa cada item de dados em palavras.
# Retorna uma nova linha para cada palavra, na coluna word
words = clean_lines.select(
   explode(
       split(clean_lines.line_clean, " ")
   ).alias("word")
)

#  .withColumn("timestamp",current_timestamp()) \
  #.withWatermark("timestamp", "5 seconds") \

# Contagem de palavras acumulativa
wordCounts = words \
  .groupBy("word") \
  .count()

query = wordCounts.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

#não dá para agregar no lote, só dá para agregar em cima da tabela como um todo

query.awaitTermination(180)


In [ ]:
!rm /content/stream_input/*


In [ ]:
!spark-submit wc_per_batch.py

# Streaming via Rede

In [ ]:
!pip install colab-xterm #permite abrir um terminal dentro do colab, o problema é que ele é executado e fecha
%load_ext colabxterm

**Instruções:** Antes de iniciar o stream, executar o comando ```nc localhost -l 9998 < '/content/drive/MyDrive/gut-pt/small/u-27350-8'``` no xterm

In [ ]:
%xterm #aqui abre o terminal e não fecha após execução do comando


# Spark Streaming via Rede

In [ ]:
%%file wc_net.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Big Data Streaming') \
    .getOrCreate()

spark.sparkContext.setLogLevel('WARN')

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9998) \
    .load()

clean_lines = lines \
    .withColumn('line_lower', lower(col('value'))) \
    .select(regexp_replace('line_lower', r'[^a-zà-ù ]', '').alias('line_clean'))

# Separa cada item de dados em palavras.
# Retorna uma nova linha para cada palavra, na coluna word
words = clean_lines.select(
   explode(
       split(clean_lines.line_clean, " ")
   ).alias("word")
)

#  .withColumn("timestamp",current_timestamp()) \
  #.withWatermark("timestamp", "5 seconds") \

# Contagem de palavras acumulativa
wordCounts = words \
  .groupBy("word") \
  .count()

query = wordCounts.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination(180)


In [ ]:
!spark-submit  wc_net.py